In [ ]:
import geopandas as gpd
import rasterio
import pandas as pd

# Define the boundary of your region
xmin = 718204                                              #THESE ARE EXTENTS FROM RASTER. ( This is an example). To get the extents the rasters have to reprojected to UTM projection.
xmax = 798147
ymin = 3013022
ymax = 3097902

# Create a point shapefile withing the given extent at X meter interval. For example here 300 meter interval was taken.
points = []
x_interval = 300
y_interval = 300
for x in range(xmin, xmax, x_interval):
    for y in range(ymin, ymax, y_interval):
        points.append((x, y))

# Convert points to a geopandas dataframe
df = pd.DataFrame(points, columns=['x', 'y'])
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x, df.y))

# Extract raster data for each point
raster_file = '....path..../Raster_Data.tif'
with rasterio.open(raster_file) as src:
    values = []
    for idx, row in gdf.iterrows():
        x, y = row['geometry'].x, row['geometry'].y
        row, col = src.index(x, y)
        value = src.read(1, window=((row, row+1), (col, col+1)))[0][0]
        values.append(value)
    gdf['value'] = values

# Convert UTM coordinates to latitude and longitude
from pyproj import Proj, transform
inProj = Proj(init='epsg:32646')
outProj = Proj(init='epsg:4326')
gdf['lon'], gdf['lat'] = transform(inProj, outProj, gdf.x.values, gdf.y.values)

# Create a dataframe with columns (Longitude, Latitude, data of raster)
result = gdf[['lon', 'lat', 'value']]

# Dataframe to csv.
result.to_csv('....path..../Csv_Data.csv')